In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
import os
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder

In [ ]:
print(os.listdir("../input"))

In [ ]:
pd.read_csv("../input/sample_submission.csv").head()

Data loading

In [ ]:
trainDF = pd.read_csv("../input/train.csv")
testDF = pd.read_csv("../input/test.csv")

#trainDF.head()
testDF.head()

Preprocessing 

In [ ]:
trainDF = trainDF[["description","price","title","city","category_name","deal_probability"]].dropna(axis=0, how='any')[0:100000]
testDF = testDF[["item_id","price","title","city","category_name","description"]].fillna(0)

max_features = 20000
maxlen = 200

tokenizer = Tokenizer(num_words=max_features)

# Description 
tokenizer.fit_on_texts(trainDF["description"])

trainDF["description"] = tokenizer.texts_to_sequences(trainDF["description"])
testDF["description"]  = tokenizer.texts_to_sequences(testDF["description"])

train_x = pad_sequences(trainDF["description"], maxlen=maxlen)
test_x = pad_sequences(testDF["description"], maxlen=maxlen)

# Title 
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(trainDF["title"])

trainDF["title"] = tokenizer.texts_to_sequences(trainDF["title"])
testDF["title"]  = tokenizer.texts_to_sequences(testDF["title"])

train_title = pad_sequences(trainDF["title"], maxlen=maxlen)
test_title = pad_sequences(testDF["title"], maxlen=maxlen)

# Category
trainDF['category_name'], mapping_index_category = pd.Series(trainDF['category_name']).factorize()
reverseCatMap = {mapping_index_category[i]: i for i in range(0, len(mapping_index_category))}
testDF['category_name'] = testDF['category_name'].map(reverseCatMap)

# City
trainDF['city'], mapping_index_city = pd.Series(trainDF['city']).factorize()
reverseCityMap = {mapping_index_city[i]: i for i in range(0, len(mapping_index_city))}
testDF['city'] = testDF['city'].map(reverseCityMap)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,Concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 128
inp = Input(shape=(maxlen, )) #maxlen=200 as defined earlier
x = Embedding(max_features, embed_size)(inp)
x = LSTM(100, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)

embed_size = 128
titleInp = Input(shape=(maxlen, )) #maxlen=200 as defined earlier
y = Embedding(max_features, embed_size)(titleInp)
y = LSTM(100, return_sequences=True,name='lstm_layer2')(y)
y = GlobalMaxPool1D()(y)
y = Dropout(0.1)(y)

x = Concatenate(axis=-1)([x,y])

x = Dense(300, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.1)(x)

priceInp = Input(shape=(1, ))
catInp = Input(shape=(1, ))
cityInp = Input(shape=(1, ))

x = Concatenate(axis=-1)([x,priceInp,catInp,cityInp])
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)


x = Dense(1, activation="sigmoid")(x)

model = Model(inputs=[inp,titleInp,priceInp,catInp,cityInp], outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

In [ ]:
model.fit([train_x,train_title,trainDF["price"],trainDF["category_name"],trainDF["city"]],trainDF[["deal_probability"]],epochs=5, batch_size=300,validation_split=0.1)

In [ ]:
#modelPred = model.predict([test_x[0:10],testDF["price"]],batch_size=300).reshape(10)
modelPred = model.predict([test_x,testDF["price"],testDF["category_name"]],batch_size=300).reshape(len(test_x))

In [ ]:
data_to_submit = pd.DataFrame.from_items([
    ('item_id',testDF["item_id"]),
    ('deal_probability', pd.Series(modelPred))])

data_to_submit.to_csv('csv_to_submit.csv', index = False)

#data_to_submit.head()

In [ ]:
#resDF['category_name'] = resDF['category_name'].astype('category')
#resDF['category_name'], mapping_index_category = pd.Series(resDF['category_name']).factorize()
#resDF['city'], mapping_index_city = pd.Series(resDF['city']).factorize()
#resDF = resDF.dropna(axis=0, how='any')
#resDF["description"] = resDF["description"].map(lambda z: len(z))
#resDF[["price"]] /= resDF[["price"]].max()
#resDF[["price","category_name","description"]] /= resDF[["price","category_name","description"]].max()
#plt.matshow(trainDF[["price","category_name","city","deal_probability"]].corr())
#plt.show()
#plt.subplots(figsize=(10,10))
#corr = resDF.corr()
#sns.heatmap(corr, 
#            xticklabels=corr.columns.values,
#            yticklabels=corr.columns.values,)
#enc = OneHotEncoder()
#catList = [ [i] for i in range(0, len(mapping_index_category))]
#enc.fit(catList)

#train_cat_onehot = enc.transform(np.array(trainDF['category_name'].data).reshape(-1,1)).toarray()
#test_cat_onehot = enc.transform(np.array(testDF['category_name'].data).reshape(-1,1)).toarray()
#trainDF['category_name'] = trainDF['category_name'].map(lambda x : enc.transform([[x]]).toarray())


#testDF['category_name'] = testDF['category_name'].map(lambda x : enc.transform([[reverseCatMap[x]]]).toarray())